# Working with the GEMS Weather API
This notebook demonstrates access patterns for the GEMS Weather API using a few example endpoints out of the many that are available. Please see [the documentation](https://exchange-1.gems.msi.umn.edu/weather/v2/docs) for a complete list of endpoints. Available datasets include:
- Alerts
- **Current Weather Data**
- 16 day / daily Forecast
- Air Quality Forecast
- Current Air Quality
- Historical Air Quality
- 240 hour / hourly Forecast
- **Daily Historical Weather Data**
- Hourly Historical Weather Data
- Sub-Hourly Historical Weather Data
- Forecast Degree Day API
- Forecast Solar Irradiance API
- Historical Degree Day API
- Historical Solar Irradiance API
- Historical Weather Energy API

Note that the access patterns for this API differ from most other GEMS APIs. Users are encouraged to use the documentation (linked above) as a guide.

## Setup
### Create a Session
We use a request session and an API key to interact with the APIs. Note that this cell may need to be re-run if there is a long gap between sent requests.
Note that we have a `api_key.py` file in the same directory as this notebook. The file contains only the below line.
```
key = 'SECRET'
```

In [1]:
from requests import Session
import sys
sys.path.append('..')
from api_key import key
import pandas as pd

s = Session()
s.headers.update({'apikey': key})

api_url = 'https://exchange-1.gems.msi.umn.edu/weather/v2'

## Accessing Current Weather Data
For documentation on returned objects see the `Schemas` section of the documentation page linked above. See the `CurrentObs` object specifically for current weather.
### Using points
Obtain current weather data for a few `[lon, lat]` points and use them to create a DataFrame.

In [2]:
points = [
    [-93.2, 44.9],
    [-74.0, 40.7],
    [-115.1, 36.1],
    [-81.7, 26.1],
    [-122.3, 47.6]
]

data = []
for pt in points:
    params = {
        'lon': pt[0],
        'lat': pt[1]
    }
    with s.get(f'{api_url}/current', params=params) as req:
        weather = req.json()

    weather = weather['data'][0]
    data.append(weather)

pd.json_normalize(data)

,app_temp,aqi,city_name,clouds,country_code,datetime,dewpt,dhi,dni,elev_angle,...,ts,uv,vis,wind_cdir,wind_cdir_full,wind_dir,wind_spd,weather.code,weather.icon,weather.description
0,20.9,54,Mendota,4,US,2024-05-14:19,6.9,119.58,915.18,61.92,...,1715713624,8.673661,16,ENE,east-northeast,72,4.179688,800,c01d,Clear sky
1,20.9,29,New York City,94,US,2024-05-14:19,13.9,116.47,894.38,55.07,...,1715713500,2.664641,16,SSW,south-southwest,200,1.500000,804,c04d,Overcast clouds
2,31.2,62,Winchester,93,US,2024-05-14:17,-1.4,121.92,935.91,70.96,...,1715713625,3.747489,16,E,east,92,1.379883,804,c04d,Overcast clouds
3,43.5,63,Lely Resort,6,US,2024-05-14:19,26.3,122.54,922.35,66.55,...,1715713200,9.601019,16,SE,southeast,141,1.340000,800,c01d,Clear sky
4,18.3,42,Seattle,80,US,2024-05-14:19,9.9,118.44,903.66,58.30,...,1715713625,3.681744,16,NW,northwest,316,3.529297,803,c03d,Broken clouds


### Using city and state names for USA

In [3]:
city_states = [
    ['minneapolis', 'minnesota'],
    ['new york city', 'new york'],
    ['las vegas', 'nevada'],
    ['naples', 'florida'],
    ['seattle', 'washington']
]

data = []
for cs in city_states:
    params = {
        'city': cs[0],
        'state': cs[1],
        'country': 'us'
    }
    with s.get(f'{api_url}/current', params=params) as req:
        weather = req.json()

    weather = weather['data'][0]
    data.append(weather)

pd.json_normalize(data)

,app_temp,aqi,city_name,clouds,country_code,datetime,dewpt,dhi,dni,elev_angle,...,ts,uv,vis,wind_cdir,wind_cdir_full,wind_dir,wind_spd,weather.code,weather.icon,weather.description
0,19.0,61,Minneapolis,3,US,2024-05-14:19,5.2,119.58,915.18,61.92,...,1715713680,8.735209,16,NNE,north-northeast,14,3.510000,800,c01d,Clear sky
1,21.0,24,New York City,94,US,2024-05-14:19,13.8,116.47,894.38,55.07,...,1715713200,2.664641,16,SSW,south-southwest,213,2.810000,804,c04d,Overcast clouds
2,30.2,52,Las Vegas,88,US,2024-05-14:19,1.5,121.92,935.59,70.83,...,1715713200,4.100484,16,NE,northeast,42,0.380000,803,c03d,Broken clouds
3,39.0,63,Naples,0,US,2024-05-14:19,24.9,122.56,922.47,66.61,...,1715712900,10.026357,14,SW,southwest,220,5.140000,800,c01d,Clear sky
4,17.8,42,Seattle,80,US,2024-05-14:19,10.9,118.44,903.66,58.30,...,1715713688,3.681744,16,WNW,west-northwest,302,4.539062,803,c03d,Broken clouds


## Accessing Daily Historical Weather Data
For documentation on returned objects see the `Schemas` section of the documentation page linked above. See the `HistoryDayObj` object specifically for current weather.
### Using a point
Obtain daily weather data for January 2019 for a `[lon, lat]` point and create a DataFrame. Note that this follows the same approach as with the Current Weather Data endpoint, but uses the `/history/daily` endpoint instead.

In [4]:
point = [-122.3, 47.6]
start_date = '2019-01-01'
end_date = '2019-02-1'

params = {
    'lon': point[0],
    'lat': point[1],
    'start_date': start_date,
    'end_date': end_date
}
with s.get(f'{api_url}/history/daily', params=params) as req:
    data = req.json()

historical_weather = pd.json_normalize(data['data'])
historical_weather.head()

,clouds,datetime,dewpt,dhi,dni,ghi,max_dhi,max_dni,max_ghi,max_temp,...,solar_rad,t_dhi,t_dni,t_ghi,t_solar_rad,temp,ts,wind_dir,wind_gust_spd,wind_spd
0,71,2019-01-01,-0.3,20,179,63,79,686,299,6.7,...,23,491,4296,1509,549,2.6,1546329600,147,3.2,1.0
1,78,2019-01-02,0.0,21,180,63,79,687,300,8.3,...,15,493,4316,1520,369,4.3,1546416000,145,8.8,3.0
2,99,2019-01-03,6.2,21,181,64,79,689,302,13.3,...,11,496,4337,1533,252,9.6,1546502400,202,15.6,4.4
3,86,2019-01-04,5.7,21,182,64,79,690,304,11.0,...,47,498,4358,1546,1119,9.6,1546588800,160,11.4,4.6
4,75,2019-01-05,4.8,21,183,65,80,691,306,10.0,...,16,501,4381,1560,390,7.5,1546675200,242,7.2,1.6


### Using a city and state name for USA
Note that this follows the same approach as with the Current Weather Data endpoint, but uses the `/history/daily` endpoint instead.

In [5]:
city_state = ['seattle', 'washington']
start_date = '2019-01-01'
end_date = '2019-02-1'

params = {
    'city': city_state[0],
    'state': city_state[1],
    'country': 'us',
    'start_date': start_date,
    'end_date': end_date
}
with s.get(f'{api_url}/history/daily', params=params) as req:
    data = req.json()

historical_weather = pd.json_normalize(data['data'])
historical_weather.head()

,clouds,datetime,dewpt,dhi,dni,ghi,max_dhi,max_dni,max_ghi,max_temp,...,solar_rad,t_dhi,t_dni,t_ghi,t_solar_rad,temp,ts,wind_dir,wind_gust_spd,wind_spd
0,71,2019-01-01,-0.3,20,179,63,79,686,299,6.7,...,23,491,4296,1509,549,2.6,1546329600,147,3.2,1.0
1,78,2019-01-02,0.0,21,180,63,79,687,300,8.3,...,15,493,4316,1520,369,4.3,1546416000,145,8.8,3.0
2,99,2019-01-03,6.2,21,181,64,79,689,302,13.3,...,11,496,4337,1533,252,9.6,1546502400,202,15.6,4.4
3,86,2019-01-04,5.7,21,182,64,79,690,304,11.0,...,47,498,4358,1546,1119,9.6,1546588800,160,11.4,4.6
4,75,2019-01-05,4.8,21,183,65,80,691,306,10.0,...,16,501,4381,1560,390,7.5,1546675200,242,7.2,1.6
